# **Players FIFA Attributes**

#### *Set-up & Functions*

Packages Import

In [1]:
# Import needed packages
import pandas as pd
import numpy as np
import collections.abc
from tqdm.notebook import trange, tqdm
from unidecode import unidecode
from itertools import groupby
import os
import glob
from datetime import datetime

# Change Pandas rows and columns' options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', None)

Generic Import Functions

In [2]:
# Turn a nested dictionary into a flattened dictionary
def flatten(d, parent_key='', sep='_'): 
    """This function turns a nested dictionary into a flattened dictionary.
    d: nested dictionary
    """
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [3]:
def check_columns_equal(df, reg=None, add_columns=None, to_print=True):
    """This functions checks, given a DataFrame and a regular expression, if all columns containing the specific re are equals (i.e. have the same values), printing the test results and returning a boolean (True if all considered columns are equals, else False). Possibility to add either a list or a string indicating a group of additional columns or a single column (add_columns).
    dataframe: DataFrame
    reg (Default: None): regular expression for columns filtering (string)
    add_columns (Default: None): either a list or a string indicating a group of columns or a single column.
    print (Default: True): If True it print result messages, while it does not when False. Warning messages (if any) are always printed.
    """
    fix_cols = []
    # If reg is not None, it has to be a string type
    if reg is not None: 
        if isinstance(reg, str): fix_cols = df.filter(regex=reg).columns.tolist()
        else: 
            print('reg supports only string type. ', type(reg), ' is not supported!') 
            return
    # If reg is not None, it has to be either a list or string type
    if add_columns is not None:
        if isinstance(add_columns, list): fix_cols = fix_cols + add_columns
        elif isinstance(add_columns, str): fix_cols.append(add_columns) 
        else: 
            print('add_columns supports either list or string types. ', type(add_columns), ' is not supported!')
            return 
    # Check if all specified columns are equal + consider to print or not
    ch = {(col1, col2): df[col1].equals(df[col2]) if len(fix_cols) > 1 else \
        print('Single columns are always equal to themselves!') for col1 in fix_cols for col2 in fix_cols}
    if len(ch.values()) != sum(ch.values()):
        if to_print:
            print('PROBLEM: not all FIX cols are equal')
            for k, v in ch.items():
                if not v: print(k, ' are different!!!')
        return False 
    else: 
        if to_print: print('OK: All Fix cols have same obs.')
        return True

In [4]:
def replace_names_using_dictionary(series, dict_old_to_new, reg=False):
    """This functions, given a series and a dictionary (where old names are keys and new names are values), replaces the old names with the new names inside the series values, and return the modified series.
    series: pandas Series type
    dict_old_to_new: dictionary containing old names as keys and new names as values (old_name: new_name)
    """
    for old, new in dict_old_to_new.items():
        series = series.str.replace(old, new, regex=reg)
    return series

In [5]:
def different_values_series(series1, series2, reverse=False):
    """This functions, given a two series (series1 and series2), prints the difference between the values in series1 and series2. If reverse is True, the function also prints the difference between series2 and series1.
    series1: pandas Series type, first
    series2: pandas Series type, second
    reverse (Default: False): if True prints also the opposite order operation
    """
    s1_diff_s2 = (set(series1)-set(series2))
    print('------ Test ------  Names in',series1.name,'but not in',series2.name,': ')
    print('None!!') if not s1_diff_s2 else print(s1_diff_s2)
    if reverse:
        s2_diff_s1 = (set(series2)-set(series1))
        print('------ Test ------  Names in',series2.name,'but not in',series1.name,': ')
        print('None!!') if not s2_diff_s1 else print(s2_diff_s1)

In [6]:
def clean_player_data(df):
    """Given a Dataframe of players data, produces some pre-cleaning operations: drop non-wanted observations and features, convert date column format, reorder columns and sort observations, conform all numeric variables to integers. 
    Finally returns the cleaned DataFrame.
    Parameters:
    - df: Pandas Dataframe of players data"""
    
    # Drop observations from FIFA10 and FIFA 11 (to avoid problems with different pages' formats). Marking observation were discontinued in FIFA23 so if we want to consider also them we should not consider Marking attribute
    df = df[(df['ObservationDate']!='Sept. 1, 2009') & (df['ObservationDate']!='Sept. 1, 2010') & (~df['Club'].isna()) & (~df['Value'].isna()) & (~df['Wage'].isna()) & (~df['Marking'].isna())] 
    # Drop useless columns and Composure (too many NA)
    df.drop(['Unnamed: 0', 'Onloanfrom', 'OnloanfromFCIngolstadt', 'OnloanfromFCSchalke', 'OnloanfromHannover', 'Tackling', 'Handling', 'Reflexes', 'Composure', 'WorkRate'], axis=1, inplace=True)
    # Change ObservationDate to Date Format
    df['ObservationDate'] = df['ObservationDate'].str.replace('Sept', 'Sep')
    df['ObservationDate'] = df['ObservationDate'].str.replace('April', 'Apr.')
    df['ObservationDate'] = df['ObservationDate'].str.replace('June', 'Jun.')
    df['ObservationDate'] = df['ObservationDate'].str.replace('March', 'Mar.')
    df['ObservationDate'] = df['ObservationDate'].str.replace('May', 'May.')
    df['ObservationDate'] = df['ObservationDate'].str.replace('July', 'Jul.')
    df['ObservationDate'] = df['ObservationDate'].map(lambda x: datetime.strptime(x, '%b. %d, %Y'))
    # Create Dummy for Players that play for National Teams
    df['PlayNational'] = np.where(df['NationalTeam'].isna(), 0, 1)
    # Reorder columns order + Sort by Date and Name (also reset index)
    df = df[['ObservationDate', 'Name', 'KitNumber', 'FavoritePosition', 'Age', 'Height', 'Weight', 'Club', 'PlayNational', 'NationalTeam', 'Value', 'Wage', 'BallControl', 'Marking', 'Dribbling', 'SlideTackle', 'StandTackle', 'Aggression', 'Reactions', 'Att.Position', 'Interceptions', 'Vision', 'Crossing', 'ShortPass', 'LongPass', 'Acceleration', 'Stamina', 'Strength', 'Balance', 'SprintSpeed', 'Agility', 'Jumping', 'Heading', 'ShotPower', 'Finishing', 'LongShots', 'Curve',  'FKAcc.', 'Penalties', 'Volleys', 'GKPositioning', 'GKDiving', 'GKHandling', 'GKKicking', 'GKReflexes', 'Overall', 'Potential']]
    df = df.sort_values(by = ['ObservationDate', 'Name']).reset_index(drop = True)
    # Change all numeric columns to Integer (to avoid Float type)
    m = df.select_dtypes(np.number)
    df.loc[:, m.columns] = df.loc[:, m.columns].astype(int)
    # Clean all values of attributes' columns (keeping only last 2 digits)
    for i in df.loc[:, 'BallControl':].columns.tolist(): 
        df.loc[:, i] = df.loc[:, i].astype(str).str[-2:].astype(int)
    return df

## *Create Lineup DataFrame*

Import necessary datasets

In [7]:
# Import data
leagues_more = pd.read_csv('../../Data/From_Collection/Match&Odds/leagues_more.csv', low_memory=False).set_index('id')
leagues_static = pd.read_csv('../../Data/From_Collection/Match&Odds/leagues_static.csv', low_memory = False).set_index('id')
# Merge data + Prepare columns
df = leagues_more.merge(leagues_static.loc[:,['localTeam_name','visitorTeam_name']], how='left', left_index=True, right_index=True)
df = df.loc[:, ['localteam_id','visitorteam_id','time_starting_at_date','lineup','localTeam_name','visitorTeam_name']]
df['time_starting_at_date'] = pd.to_datetime(df['time_starting_at_date'], format = '%Y-%m-%d')
# Consider only observations from 2016/2017 season onwards
df = df.loc[(df['time_starting_at_date'] > '2016-07-01') & (~df['localTeam_name'].isna()) & (~df['visitorTeam_name'].isna()), :]
print('DataFrame shape before dropping incomplete lineups observations: ', df.shape)
# Drop matches with incomplete lineups data (when number of total players is different from 22) 
incomplete_lineups = [df.index[line] for line in range(len(df)) if len(eval(df['lineup'].iloc[line])) != 22]
print('Number of matches with incomplete lineup data: ', len(incomplete_lineups), \
    '\nDifferent incomplete observations: ', set(df.loc[incomplete_lineups, 'lineup']))
df = df.loc[set(df.index) - set(incomplete_lineups), :] #Drop incomplete obs.
print('DataFrame shape after dropping incomplete lineups observations: ', df.shape)
df.head(3)

DataFrame shape before dropping incomplete lineups observations:  (10638, 6)
Number of matches with incomplete lineup data:  101 
Different incomplete observations:  {'[]'}
DataFrame shape after dropping incomplete lineups observations:  (10537, 6)


,localteam_id,visitorteam_id,time_starting_at_date,lineup,localTeam_name,visitorTeam_name
id,,,,,,
10420224,398,97,2018-12-09,"[{'team_id': 398, 'fixture_id': 10420224, 'pla...",Parma,Chievo
10420225,2714,109,2018-12-09,"[{'team_id': 2714, 'fixture_id': 10420225, 'pl...",Sassuolo,Fiorentina
10420226,346,708,2018-12-09,"[{'team_id': 346, 'fixture_id': 10420226, 'pla...",Udinese,Atalanta


Create only_lineup dataset 

In [8]:
# Create new DataFrame from the lineup feature (after eval() is a list of players' dictionaries). For the new DataFrame structure, it is optimal to consider separately players from the home and away team. By grouping players (to create players' columns) for their position in the formation (and filling missing values), we can obtain for each feature type 11 columns for home players (1 for each) and 11 for away players.
all_matches, no_G_pl_set = [], set(list(range(2,12))) # all_matches to store data, no_G_pl_set possible formation pos. outside GK

for i in trange(len(df)):
    match_FT, form_pos_home_store, form_pos_away_store = {}, [], [] # To store data produced
    single_match = eval(df['lineup'].iloc[i])
    for pl in single_match:
        # First If is for Home teams, while second is for Away teams (same process except for the final prefix and storing element)
        if pl['fixture_id'] == df.index[i] and pl['team_id'] == df['localteam_id'].iloc[i]:
            if pl['formation_position'] is None:
                pl['formation_position'] = 1 if pl['position'] == 'G' else list(no_G_pl_set - set(form_pos_home_store))[0]
            form_pos_home_store.append(pl['formation_position'])
            match_FT['home_' + 'pl' + str(pl['formation_position'])] = pl
        elif pl['fixture_id'] == df.index[i] and pl['team_id'] == df['visitorteam_id'].iloc[i]:
            if pl['formation_position'] is None:
                pl['formation_position'] = 1 if pl['position'] == 'G' else list(no_G_pl_set - set(form_pos_away_store))[0]
            form_pos_away_store.append(pl['formation_position'])
            match_FT['away_' + 'pl' + str(pl['formation_position'])] = pl
        # Check for inconsistency in fixtures IDs 
        else: print('Error in row: ', i, '(Fixture id is not compatible!!!)')
    # Store flatten data into all_matches     
    all_matches.append(flatten(match_FT))
    
# Transform only lineup data from a list of dictionaries in a DataFrame
only_lineup = pd.DataFrame(all_matches).dropna(how = 'all')

  0%|          | 0/10537 [00:00<?, ?it/s]

Merge only_lineup with some match infos

In [9]:
# If Fixture ID columns are equal for each row, merge dataframe containing only lineups and important columns from original df
if check_columns_equal(only_lineup,'fixture_id$'): 
    only_lineup = only_lineup.rename(columns={'home_pl1_fixture_id': 'id'}).set_index('id')
    lineups_df = pd.merge(df.loc[:, df.columns != 'lineup'], only_lineup, how='inner', left_index=True, right_index=True)
# Rename columns + sort values by date
lineups_df = lineups_df.rename(columns={'localteam_id':'home_id', 'visitorteam_id':'away_id', 'time_starting_at_date':'date',\
    'localTeam_name':'home_name', 'visitorTeam_name':'away_name'}).sort_values(by='date', ascending=True)
lineups_df.tail(3)

OK: All Fix cols have same obs.


,home_id,away_id,date,home_name,away_name,home_pl1_team_id,home_pl1_player_id,home_pl1_player_name,home_pl1_number,home_pl1_position,home_pl1_additional_position,home_pl1_formation_position,home_pl1_posx,home_pl1_posy,home_pl1_captain,home_pl1_type,home_pl1_stats_shots_shots_total,home_pl1_stats_shots_shots_on_goal,home_pl1_stats_goals_scored,home_pl1_stats_goals_assists,home_pl1_stats_goals_conceded,home_pl1_stats_goals_owngoals,home_pl1_stats_goals_team_conceded,home_pl1_stats_fouls_drawn,home_pl1_stats_fouls_committed,home_pl1_stats_cards_yellowcards,home_pl1_stats_cards_redcards,home_pl1_stats_cards_yellowredcards,home_pl1_stats_passing_total_crosses,home_pl1_stats_passing_crosses_accuracy,home_pl1_stats_passing_passes,home_pl1_stats_passing_accurate_passes,home_pl1_stats_passing_passes_accuracy,home_pl1_stats_passing_key_passes,home_pl1_stats_dribbles_attempts,home_pl1_stats_dribbles_success,home_pl1_stats_dribbles_dribbled_past,home_pl1_stats_duels_total,home_pl1_stats_duels_won,home_pl1_stats_other_aerials_won,home_pl1_stats_other_punches,home_pl1_stats_other_offsides,home_pl1_stats_other_saves,home_pl1_stats_other_inside_box_saves,home_pl1_stats_other_pen_scored,home_pl1_stats_other_pen_missed,home_pl1_stats_other_pen_saved,home_pl1_stats_other_pen_committed,home_pl1_stats_other_pen_won,home_pl1_stats_other_hit_woodwork,home_pl1_stats_other_tackles,home_pl1_stats_other_blocks,home_pl1_stats_other_interceptions,home_pl1_stats_other_clearances,home_pl1_stats_other_dispossesed,home_pl1_stats_other_minutes_played,home_pl1_stats_rating,home_pl2_team_id,home_pl2_fixture_id,home_pl2_player_id,home_pl2_player_name,home_pl2_number,home_pl2_position,home_pl2_additional_position,home_pl2_formation_position,home_pl2_posx,home_pl2_posy,home_pl2_captain,home_pl2_type,home_pl2_stats_shots_shots_total,home_pl2_stats_shots_shots_on_goal,home_pl2_stats_goals_scored,home_pl2_stats_goals_assists,home_pl2_stats_goals_conceded,home_pl2_stats_goals_owngoals,home_pl2_stats_goals_team_conceded,home_pl2_stats_fouls_drawn,home_pl2_stats_fouls_committed,home_pl2_stats_cards_yellowcards,home_pl2_stats_cards_redcards,home_pl2_stats_cards_yellowredcards,home_pl2_stats_passing_total_crosses,home_pl2_stats_passing_crosses_accuracy,home_pl2_stats_passing_passes,home_pl2_stats_passing_accurate_passes,home_pl2_stats_passing_passes_accuracy,home_pl2_stats_passing_key_passes,home_pl2_stats_dribbles_attempts,home_pl2_stats_dribbles_success,home_pl2_stats_dribbles_dribbled_past,home_pl2_stats_duels_total,home_pl2_stats_duels_won,home_pl2_stats_other_aerials_won,home_pl2_stats_other_punches,home_pl2_stats_other_offsides,home_pl2_stats_other_saves,home_pl2_stats_other_inside_box_saves,home_pl2_stats_other_pen_scored,home_pl2_stats_other_pen_missed,home_pl2_stats_other_pen_saved,home_pl2_stats_other_pen_committed,home_pl2_stats_other_pen_won,home_pl2_stats_other_hit_woodwork,home_pl2_stats_other_tackles,home_pl2_stats_other_blocks,home_pl2_stats_other_interceptions,home_pl2_stats_other_clearances,home_pl2_stats_other_dispossesed,home_pl2_stats_other_minutes_played,home_pl2_stats_rating,home_pl3_team_id,home_pl3_fixture_id,home_pl3_player_id,home_pl3_player_name,home_pl3_number,home_pl3_position,home_pl3_additional_position,home_pl3_formation_position,home_pl3_posx,home_pl3_posy,home_pl3_captain,home_pl3_type,home_pl3_stats_shots_shots_total,home_pl3_stats_shots_shots_on_goal,home_pl3_stats_goals_scored,home_pl3_stats_goals_assists,home_pl3_stats_goals_conceded,home_pl3_stats_goals_owngoals,home_pl3_stats_goals_team_conceded,home_pl3_stats_fouls_drawn,home_pl3_stats_fouls_committed,home_pl3_stats_cards_yellowcards,home_pl3_stats_cards_redcards,home_pl3_stats_cards_yellowredcards,home_pl3_stats_passing_total_crosses,home_pl3_stats_passing_crosses_accuracy,home_pl3_stats_passing_passes,home_pl3_stats_passing_accurate_passes,home_pl3_stats_passing_passes_accuracy,home_pl3_stats_passing_key_passes,home_pl3_stats_dribbles_attempts,home_pl3_stats_dribbles_success,home

Clean lineups_df

In [10]:
# Drop all columns that have only NAs by both axes
lineups_df = lineups_df.dropna(axis=1, how='all').dropna(axis=0, how='all')
# Drop all columns names matching specific regular expressions - After checking for consistency (all filtered columns must be equal)
dict_check_and_drop = {'_fixture_id$': [], '^home.*_team_id$': 'home_id', '^away.*_team_id$': 'away_id'}
for regex_string, additional_columns in dict_check_and_drop.items():
    if check_columns_equal(df=lineups_df, reg=regex_string, add_columns = additional_columns, to_print=False): 
        lineups_df.drop(lineups_df.filter(regex=regex_string).columns, axis = 1, inplace=True)
# Drop all columns matching specific regular expressions - No check needed
for column_type in ['_player_id$', '_formation_position$', '_posx$', '_posy$', '_captain$', '_type$', 'stats']:
    lineups_df.drop(lineups_df.filter(regex=column_type).columns, axis=1, inplace=True)
# Transfor all numeric columns into Integer + downcast (optimize for memory)
int_cols = lineups_df.select_dtypes(np.number).columns
lineups_df[int_cols] = lineups_df[int_cols].apply(pd.to_numeric, errors='coerce', downcast='integer')
lineups_df.head(3)

,home_id,away_id,date,home_name,away_name,home_pl1_player_name,home_pl1_number,home_pl1_position,home_pl2_player_name,home_pl2_number,home_pl2_position,home_pl3_player_name,home_pl3_number,home_pl3_position,home_pl4_player_name,home_pl4_number,home_pl4_position,home_pl5_player_name,home_pl5_number,home_pl5_position,home_pl6_player_name,home_pl6_number,home_pl6_position,home_pl7_player_name,home_pl7_number,home_pl7_position,home_pl8_player_name,home_pl8_number,home_pl8_position,home_pl9_player_name,home_pl9_number,home_pl9_position,home_pl10_player_name,home_pl10_number,home_pl10_position,home_pl11_player_name,home_pl11_number,home_pl11_position,away_pl1_player_name,away_pl1_number,away_pl1_position,away_pl2_player_name,away_pl2_number,away_pl2_position,away_pl3_player_name,away_pl3_number,away_pl3_position,away_pl4_player_name,away_pl4_number,away_pl4_position,away_pl5_player_name,away_pl5_number,away_pl5_position,away_pl6_player_name,away_pl6_number,away_pl6_position,away_pl7_player_name,away_pl7_number,away_pl7_position,away_pl8_player_name,away_pl8_number,away_pl8_position,away_pl9_player_name,away_pl9_number,away_pl9_position,away_pl11_player_name,away_pl11_number,away_pl11_position,away_pl10_player_name,away_pl10_number,away_pl10_position
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
251728,430,591,2016-08-12,Bastia,Paris Saint Germain,J. Leca,16,G,F. Marange,28,D,S. Squillaci,5,D,M. Peybernes,17,D,A. Djiku,23,D,Y. Cahuzac,18,M,M. Mostefa,14,M,S. Diallo,2,M,A. Saint-Maximin,6,M,L. Coulibaly,25,M,E. Crivelli,27,A,K. Trapp,1,G,David Luiz,32,D,L. Kurzawa,20,D,S. Aurier,19,D,P. Kimpembe,3,D,H. Ben Arfa,21,M,T. Motta,8,C,Á. Di María,11,M,J. Pastore,10,M,A. Rabiot,25,M,Lucas Moura,7,M
251738,6789,2919,2016-08-12,Monaco,Guingamp,D. Subašić,1,G,K. Glik,25,D,D. Sidibé,19,D,B. Mendy,23,D,Jemerson,5,D,N. Dirar,7,M,T. Bakayoko,14,M,T. Lemar,27,M,Fabinho,2,M,G. Carrillo,11,A,K. Mbappé,29,A,K. Johnsson,1,G,J. Sorbon,15,D,B. Angoua,4,D,C. Kerbrat,29,D,J. Ikoko,2,D,M. Diallo,5,M,L. Deaux,8,M,M. Coco,24,M,J. Briand,23,A,Y. Salibur,19,A,S. Privat,11,A
252926,3513,690,2016-08-13,Metz,Lille,T. Didillon,1,G,G. Milán,5,D,J. Rivierez,3,D,S. Falette,6,D,I. Balliu,25,D,R. Cohade,24,M,Y. Jouffre,8,M,C. Doukouré,10,M,F. Mollet,19,M,O. Nguette,11,M,M. Erdinç,9,A,V. Enyeama,1,G,R. Civelli,5,D,J. Palmieri,15,D,S. Corchia,2,D,A. Soumaoro,23,D,M. Amalfitano,10,M,M. Obbadi,8,M,É. Bauthéac,11,M,I. Amadou,6,M,Rony Lopes,17,M,Éder,19,A


In [11]:
if lineups_df.loc[4194259, 'home_pl2_position'] == '-': lineups_df.loc[4194259, 'home_pl2_position'] = 'D'
if lineups_df.loc[97980, 'home_pl6_position'] == '-': lineups_df.loc[97980, 'home_pl6_position'] = 'M'
if lineups_df.loc[97980, 'away_pl9_position'] == '-': lineups_df.loc[97980, 'away_pl9_position'] = 'M'
if lineups_df.loc[4194182, 'away_pl6_position'] == '-': lineups_df.loc[4194182, 'away_pl6_position'] = 'D'

## *Players attributes data*

Import data

In [12]:
# Import Data: merge data from CSVs of chunks in one dataframe or open complete csv file
try:
    joined_files = os.path.join("../../Data/From_Collection/FIFA_scraped/Player_inChunks", "Playersdata_chunk*.csv")
    joined_list = glob.glob(joined_files) # A list of all joined files is returned
    pl_weekly = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
except Exception as e:
    pl_weekly = pd.read_csv('../../Data/From_Collection/FIFA_scraped/players_weekly_complete.csv', low_memory=False)
  
# Clean + Prepare data 
players_attr = clean_player_data(df=pl_weekly)
players_attr['ObservationDate'] = pd.to_datetime(players_attr['ObservationDate'], format = '%Y-%m-%d')
players_attr = players_attr.loc[players_attr['ObservationDate'] > '2016-06-01', :]
int_cols = players_attr.select_dtypes(np.number).columns
players_attr[int_cols] = players_attr[int_cols].astype(int)
players_attr.head(2)

,ObservationDate,Name,KitNumber,FavoritePosition,Age,Height,Weight,Club,PlayNational,NationalTeam,Value,Wage,BallControl,Marking,Dribbling,SlideTackle,StandTackle,Aggression,Reactions,Att.Position,Interceptions,Vision,Crossing,ShortPass,LongPass,Acceleration,Stamina,Strength,Balance,SprintSpeed,Agility,Jumping,Heading,ShotPower,Finishing,LongShots,Curve,FKAcc.,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes,Overall,Potential
132685,2016-06-02,Aaron Cresswell,3,LB,26,170,66,West Ham United,0,NaN,3300000,35000,71,76,73,78,75,68,67,60,67,66,80,71,68,79,79,56,90,78,78,85,58,71,53,62,70,69,59,40,9,14,7,9,12,74,76
132686,2016-06-02,Aaron Hunt,14,CAM,29,183,73,Hamburger SV,0,NaN,7000000,70000,79,28,78,35,34,59,80,77,55,76,77,79,70,75,75,65,72,69,76,65,64,76,72,76,77,76,81,77,16,15,12,7,15,77,77


Change position feature into: G (Goalkeeper), D (Defender), M (Midfielder), and S (Strikers)

In [13]:
# In Lineups data
dict_pos_lineup = {'^AM$':'M','^AM-L$':'M','^AM-R$':'M','^C$':'M','^CD$':'D','^CD-L$':'D','^CD-R$':'D','^CF-L$':'S','^CF-R$':'S','^CM$':'M','^CM-L$':'M','^CM-R$':'M','^LB$':'D','^LF$':'S','^LM$':'S','^RB$':'D','^RF$':'S','^RM$':'M','^SW$':'D'}
# Loop for each position column in lineups_df
for col in lineups_df.filter(regex='\d_position$').columns:
    lineups_df.loc[:, col] = replace_names_using_dictionary(series=lineups_df.loc[:, col],dict_old_to_new=dict_pos_lineup,reg=True)

# In players attributes data (from FIFA)
dict_pos_fifa = {'^CAM$':'M','^CB$':'D','^CDM$':'M','^CF$':'S','^CM$':'M','^GK$':'G','^LB$':'D','^LM$':'M','^LW$':'M','^LWB$':'D','^RB$':'D','^RM$':'M','^RW$':'M','^RWB$':'M','^ST$':'S'}
players_attr.loc[:,'FavoritePosition'] = replace_names_using_dictionary(series=players_attr.loc[:,'FavoritePosition'],\
    dict_old_to_new=dict_pos_fifa, reg=True)

Decode Players and Teams names (by replacing special letters)

In [14]:
# Lineup data: team and players names
lineups_names = lineups_df.filter(regex='_player_name$').columns.tolist() + ['home_name', 'away_name']
for col in lineups_names:
    lineups_df.loc[:, col] = lineups_df.loc[:, col].apply(unidecode)
# Players attributes data: club and player name
for col in ['Club', 'Name']:
    players_attr.loc[:, col] = players_attr.loc[:, col].apply(unidecode)

Conforming Teams names between lineup and attributes data

In [15]:
# Teams Names from players_attr and replacement names
teams_name_repl = {'^1\. ':'','^RC ':'','^RCD ':'','^UD ':'',' UD$':'',' CF$':'','^CF ':'',' FC$':'',' Football Club':'','^AS ':'','^CD ':'',' SA$':'','^CA ':'','^ESTAC ':'',' AC$':'','^ES ':'','^Racing Club de ':'',' Atletico$':'','^OGC ':'',' FCO$':'','^Nancy.*$':'Nancy','^Celta.*$':'Celta de Vigo',' C\.F\.':'','^SM ':'','^En Avant( de | )':'','^SV ':'','^FC Ingolstadt 04$':'Ingolstadt','^FC Lorient$':'Lorient','^Stade de ':'','^Stade ':'','^FC Nantes$':'Nantes',' Alsace$':'','^Malherbe ':'','^SD Huesca$':'Huesca','^(FC |)Girondins de ':'','^Football Club de ':'','^FC Metz$':'Metz','^LOSC ':'',' Olympique$':'',' Foot(| \d{2})$':'','^Paris$':'Paris Saint Germain','^Paris Saint-Germain$':'Paris Saint Germain',' 07$':'','^SC Bastia$':'Bastia','^Hertha.*Berlin':'Hertha BSC','^SC Paderborn$':'Paderborn','^FC Nurnberg$':'Nurnberg','^FC Schalke 04$':'Schalke 04','^Deportivo$':'Deportivo La Coruna','^Deportivo de La Coruna$':'Deportivo La Coruna','^Chievo Verona$':'Chievo','Brestois 29$':'Brest','^Olympique de Marseille$':'Olympique Marseille',' HSC$':'',' Herault SC$':'','^La Spezia$':'Spezia','^Rennais$':'Rennes','^R. Sporting$':'Sporting Gijon','^Real Sporting de Gijon$':'Sporting Gijon','^Athletic Club de Bilbao$':'Athletic Club','^Spal$':'SPAL',"Borussia M'gladbach":'Borussia Monchengladbach','^Arminia Bielefeld$':'DSC Arminia Bielefeld','^R\. Valladolid$':'Real Valladolid','^Latium$':'Lazio','^Amiens SC Football$':'Amiens SC','^Salerno$':'Salernitana','^Bergamo Calcio$':'Atalanta','^TSG 1899 Hoffenheim$':'TSG Hoffenheim','^D\. Alaves':'Deportivo Alaves','^Bournemouth$':'AFC Bournemouth','^FC Bayern Munich$':'FC Bayern Munchen','^Bayern Munchen$':'FC Bayern Munchen','Sport-Club Freiburg':'SC Freiburg','^VfL Bochum$':'VfL Bochum 1848','^Real Betis Balompie$':'Real Betis','^Atletico de Madrid$':'Atletico Madrid'}
players_attr.loc[:, 'Club'] = replace_names_using_dictionary(series=players_attr.loc[:, 'Club'], dict_old_to_new=teams_name_repl, reg=True)
# Tests for names: if both OK continue with no problem
different_values_series(series1=lineups_df.loc[:, 'home_name'], series2=players_attr.loc[:, 'Club'], reverse=False)

------ Test ------  Names in home_name but not in Club : 
None!!


## *Merge Players Attributes to Lineup Data*

#### Functions

Names formatting

In [16]:
def format_name_initial(df, old_col, last_name=True):
    """"
    This function formats a name column by creating a new series for either last name of a person or the last name and the name initial. 
    
    df: dataframe
    old_col: name of the column containing people names
    including_name_initial (Default: True): if True the resulting column contains not only the last name but also the name's initial letter, if False it only contains the last name. 
    """
    if last_name:
        only_lastname = df.loc[:, old_col].str.split().str[-1].str.split('.').str[-1]
        return only_lastname
    else:
        first_name = df.loc[:, old_col].str.split().str[0]
        return first_name

Merge list of dicts to DataFrame & Test values

In [17]:
def test_merge(df, homeORaway):
    """This function tests whether the merger between DataFrames occurred correctly or not by comparing both teams names and dates. It prints how many observations are not consistently merged.
    
    df: dataframe to test
    homeORaway: either 'home' or 'away' are tested 
    """
    print('****** TEST ', homeORaway.upper(), ' ******')
    print('Name Errors = ', df[(df[homeORaway+'_lastname'] != df[homeORaway+'_Lastname']) & (df[homeORaway+'_firstname'] != df[homeORaway+'_Firstname'])].shape[0])
    print('Date Errors = ', (df[homeORaway + '_ObservationDate'] > df['date']).sum())

In [18]:
def merge_list_dicts(original_df, list_of_dicts, prefix):
    """This function merges the provided original_df with a new_df created from a provided list of dictionaries (list_of_dicts), and also uses test_merge() to test the success of the merging operation.
    
    original_df: original dataframe 
    list_of_dicts: list of dictionaries to merge with original_df
    prefix: prefix added to columns created from list_of_dicts (either 'home' or 'away', if else the function prints a warning)
    """
    new_df = pd.DataFrame(list_of_dicts).add_prefix(prefix + '_') # Create new home dataframe
    merged_df = pd.merge(original_df, new_df, how='outer', left_index=True, right_index=True) # Merge it with original_df
    test_merge(merged_df, prefix) # Test the operation
    return merged_df

Create temporary Mask of fifa data & Find closest observation (by date) + (Communication and Internal functioning between the 2 funcs)

In [19]:
def create_temp_mask_fifa(lineup_row, fifa_df, player_prefix, team_name, num):
    """This function returns a filtered dataframe. Requirements for the filtering have to became increasingly more complex if the instant and more obvious requirements are not met, the range of considered days depends on the num parameter (days considered looking at past values are double than when looking to future values - to limit for the look ahead preoblem).
    
    lineup_row: row observation from lineup dataframe
    fifa_df: FIFA players attributes dataframe
    player_prefix: prefix of the player lineup columns
    team_name: either home_name or away_name depending on the player column to consider
    num: maximum number of days to look back to (or double the maximum number of days to look ahead to)
    """ 
    # Range of considered days: preference for ObservationDate < date (double the available days than ObservationDate > date)
    days_range_ok = ((lineup_row['date'] - fifa_df['ObservationDate']).dt.days >= -(num/2)) & ((lineup_row['date'] - fifa_df['ObservationDate']).dt.days <= num)
    # Filtering by complete name + days_range
    temporary_mask = fifa_df.loc[(fifa_df['Name'] == lineup_row[player_prefix + '_player_name']) & (days_range_ok), :]
    # Filtering by Last name and same club + days_range
    if temporary_mask.shape[0] == 0: 
        temporary_mask = fifa_df.loc[(fifa_df['Club'] == lineup_row[team_name]) & (fifa_df['Lastname'] == lineup_row[player_prefix + '_lastname']) & (days_range_ok), :]
        # Filtering by kit number OR first name in the same club + days_range
        if temporary_mask.shape[0] == 0: 
            temporary_mask = fifa_df.loc[(fifa_df['Club'] == lineup_row[team_name]) & (days_range_ok) & ((fifa_df['KitNumber'] == lineup_row[player_prefix + '_number']) | (fifa_df['Firstname'] == lineup_row[player_prefix + '_firstname'])), :]
            # If still no player is found, this mask considers a random player in the same team and position (this mask is activated only when num is larger than 350) - rarely needed, is used as a sort of fillna
            if temporary_mask.shape[0] == 0 and num >= 350: 
                temporary_mask = fifa_df.loc[(fifa_df['FavoritePosition'] == lineup_row[player_prefix + '_position']) & (days_range_ok) & (fifa_df['Club'] == lineup_row[team_name]), :]
            
    return temporary_mask

In [20]:
def get_closer_date(line_row, mask_data):
    """This function, provided with a row observation from a DataFrame and a mask from another DataFrame, finds the closest observation in days and returns a dictionary containing the row data from such observation. The function tries first to find the closest day prior, if this is not possible it also looks to posterior days. 
    
    line_row: row observation from one DataFrame
    mask_data: mask from second DataFrame
    """ 
    # Considers only prior days & tries to find the minimum distance index (posterior days are given albitrary value of 999)
    time_d = [(line_row['date'].date() - j.date()).days if (line_row['date'].date() - j.date()).days >= 0 else 999 for j in (mask_data['ObservationDate'])]
    index_found = min(range(len(time_d)), key=time_d.__getitem__)
    # If prior chunk did not work!
    # It considers only posterior days & tries to find the minimum distance index (finding the max among negative numbers)
    if index_found == 0 and time_d[index_found] == 999:
        time_d_exrelegated = [(line_row['date'].date() - j.date()).days if (line_row['date'].date() - j.date()).days < 0 else -999 for j in (mask_data['ObservationDate'])]
        index_found = max(range(len(time_d_exrelegated)), key=time_d_exrelegated.__getitem__)
    # After having found the closest date index, return a dictionary containing a row observation from mask_data
    dictionary_player_obs = dict(mask_data.iloc[index_found,:])
    return dictionary_player_obs

In [21]:
def inner_loop_merger(lineup_data, attributes_data, prefix_player):
    """This function contains create_temp_mask_fifa() and get_closer_date(), and regolates their coordination inside of a loop (or apply function). Firstly, it loops for lineup_data rows, inside each loop a while loop regulates the creation of the attributes data mask (considering larger days ranges for the creation of the mask if an empty mask is returned - the quality of the mask decreases with the increase of n). When a mask is found, it uses get_closer_date() to return the closest observation and appends the generated dictionary to a list.

    lineup_data: Lineup DataFrame
    attributes_data: DataFrame of players attributes
    prefix_player: prefix of the player column to consider 
    """ 
    all_ordered = []
    team = 'home_name' if 'home' in str(prefix_player) else 'away_name' # used in create_temp_mask_fifa()
    dict_copy = lineup_data.to_dict('records') # Transform a DataFrame into dictionaries 
    # Loop over each row in lineup_data
    for row in tqdm(dict_copy):
        n = 90 # Create mask starting with n = 90
        mask_df = create_temp_mask_fifa(lineup_row=row, fifa_df=attributes_data, player_prefix=prefix_player, team_name=team, num=n)
        # If the previous mask is empty and days range not too large - add 30 days to n and try to create another mask
        while mask_df.shape[0] == 0 and n <= 420:
            n += 30
            mask_df = create_temp_mask_fifa(lineup_row=row,fifa_df=attributes_data,player_prefix=prefix_player,team_name=team,num=n)
        # Using found mask, get closer date obseravtion + append dict to list
        dict_app = get_closer_date(line_row=row, mask_data=mask_df) if mask_df.shape[0] != 0 else {}
        all_ordered.append(dict_app) # Append the row to a list
    return all_ordered

**Final Function** by using all previous functions & Tests

In [22]:
def merge_lineups_and_fifa_attributes(line_data, attr_data):
    """This function merges line_data with attr_data for each one of the provided prefixes using inner_loop_merger() to prepare the necessary data and using merge_list_dicts() to merge line_data to the returned list of dictionaries. It also carries out some preparation steps.
    
    lineup_data: Lineup DataFrame
    attr_data: DataFrame of players attributes
    """
    # Preparation players attributes DataFrame
    attr_data.drop_duplicates(inplace=True) # drop duplicates 
    attr_data.loc[:, 'Lastname'] = format_name_initial(df=attr_data, old_col='Name', last_name=True) # create lastname column
    attr_data.loc[:, 'Firstname'] = format_name_initial(df=attr_data, old_col='Name', last_name=False) # create firstname column
    # Downcast attr_data numeric columns (speeds up process)
    int_cols = attr_data.select_dtypes(np.number).columns
    attr_data[int_cols] = attr_data[int_cols].apply(pd.to_numeric, errors='coerce', downcast='integer')
    # Reset index of line_data to necessary to merge_list_dicts()
    line_data = line_data.reset_index()

    # Generate list of all prefixes
    prefixes_list = [f"{team}_pl{i}" for i in range(1, 12) for team in ['home', 'away']] 
    # Loop for each prefix
    for pref in tqdm(prefixes_list):
        # Generates new lastname and firstname columns
        line_data.loc[:, pref + '_lastname'] = format_name_initial(df=line_data, old_col=(pref + '_player_name'), last_name=True)
        line_data.loc[:, pref + '_firstname'] = format_name_initial(df=line_data, old_col=(pref + '_player_name'), last_name=False)
        # Filter only desired columns in line_data
        columns_slice = line_data.columns.tolist()[:6] + [pref + x for x in ['_player_name', '_number', '_position', '_lastname', '_firstname']]
        lineups_player = line_data.loc[:, columns_slice]
        # Use previous functions to merge data
        list_newdf = inner_loop_merger(lineup_data=lineups_player, attributes_data=attr_data, prefix_player=pref)
        print('---------- {} ----------'.format(pref))
        line_data = merge_list_dicts(original_df=line_data, list_of_dicts=list_newdf, prefix=pref)
    return line_data

In [23]:
def test_new_columns_quality(df):
    """This function provides some tests for the DataFrame resulting from the merger.
    df: DataFrame to test
    """
    na_dict, days_dict = {}, {} # Empty dictionaries
    for pref in ['home_', 'away_']: 
        for sec_pref in range(1, 12):
            full_prefix = pref + 'pl' + str(sec_pref) # full prefix
            days_differential = (df[full_prefix +'_ObservationDate'] - df['date']).dt.days # Series of days differentials
            # Print values with a larger date differential than +/- 1 Year
            print('---------- {} ----------'.format(full_prefix))
            print('N. < -365 days: ', len(days_differential < -365))
            print('N. > 365 days: ', len(days_differential > 365))
            # Days differential and NA dictionaries
            days_dict[full_prefix + '_days_diff'] = days_differential.describe().to_dict()
            na_dict[full_prefix + 'NAs by row'] = int(df[full_prefix +'_Club'].isna().sum())
    return pd.DataFrame(days_dict), pd.Series(na_dict)

#### DataFrames Merger & Final Tests

In [24]:
players_attr[players_attr.select_dtypes(['object']).columns] = players_attr.select_dtypes(['object']).apply(\
    lambda x: x.astype('category'))
    
lineups_df[lineups_df.select_dtypes(['object']).columns] = lineups_df.select_dtypes(['object']).apply(\
    lambda x: x.astype('category'))

In [25]:
merged_df = merge_lineups_and_fifa_attributes(line_data=lineups_df, attr_data=players_attr)
merged_df.head(3)

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/10537 [00:00<?, ?it/s]

---------- home_pl1 ----------
****** TEST  HOME_PL1  ******
Name Errors =  81
Date Errors =  60


  0%|          | 0/10537 [00:00<?, ?it/s]

---------- away_pl1 ----------
****** TEST  AWAY_PL1  ******
Name Errors =  97
Date Errors =  68


...

  0%|          | 0/10537 [00:00<?, ?it/s]

---------- away_pl5 ----------
****** TEST  AWAY_PL5  ******
Name Errors =  112
Date Errors =  112


  0%|          | 0/10537 [00:00<?, ?it/s]

---------- home_pl6 ----------
****** TEST  HOME_PL6  ******
Name Errors =  113
Date Errors =  100


  0%|          | 0/10537 [00:00<?, ?it/s]

In [ ]:
days_differential_summary, na_summary = test_new_columns_quality(merged_df)
days_differential_summary

In [ ]:
na_summary

In [ ]:
merged_df.to_csv('../../Data/From_Preparation/Lineup_FifaPlayersAttributes.csv', encoding='utf-8')

# Final

Functions

In [ ]:
# Groups of attributes based on the use-methods for their aggregation
entire_team_attributes = ['Acceleration', 'Age', 'Aggression', 'Agility', 'Balance',  'Crossing', 'Height', 'Jumping',  'LongPass', 'Overall',  'Potential', 'Reactions', 'ShortPass', 'SprintSpeed', 'Stamina', 'Strength', 'Value', 'Vision', 'Wage', 'Weight']
mid_att_attributes = ['Att.Position', 'BallControl', 'Curve', 'Dribbling', 'Finishing', 'Heading', 'LongShots', 'ShotPower', 'Volleys']
dif_mid_attributes = ['Marking','SlideTackle',  'StandTackle', 'Interceptions']
goalie_attributes = ['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']
only_top_attributes = ['FKAcc.', 'Penalties']
count_attributes = ['PlayNational']

def create_comp_game(game):
    game_dict = {}

    for team, team_values in game.items():
        attributes_dict = {}
        # Attributes averaged over the entire team 
        for attribute in entire_team_attributes:
            player_values = [player[attribute] for player in team_values.values()]
            mean_value = round(np.nanmean(player_values), 1)
            attributes_dict[attribute] = mean_value
        # Attributes averaged only for strikers and midfielders 
        for attribute in mid_att_attributes:
            player_values = [player[attribute] for player in team_values.values() if player['FavoritePosition'] in ['S', 'M']]
            mean_value = round(np.nanmean(player_values), 1)
            attributes_dict[attribute] = mean_value
        # Attributes averaged only for defenders
        for attribute in dif_mid_attributes:
            player_values = [player[attribute] for player in team_values.values() if player['FavoritePosition'] in ['D']]
            mean_value = round(np.nanmean(player_values), 1)
            attributes_dict[attribute] = mean_value
        # Attributes for the goalkeeper (the if-else statement tries to handle rare cases of absence of a goalie in our data)
        for attribute in goalie_attributes:
            player_values = [player[attribute] for player in team_values.values() if player['FavoritePosition'] in ['G']]
            if len(player_values) == 1: 
                attributes_dict[attribute] = player_values[0]
            else: 
                player_values = [player[attribute] for player in team_values.values()]
                max_value = np.nanmax(player_values)
                attributes_dict[attribute] = max_value
        # For attributes for free-kicks and penalties we consider only the best player of the team since it is a individual action
        for attribute in only_top_attributes:
            player_values = [player[attribute] for player in team_values.values()]
            max_value = np.nanmax(player_values)
            attributes_dict[attribute] = max_value
        # Counts the number of players that also played at the time for their national team 
        for attribute in count_attributes:
            player_values = [player[attribute] for player in team_values.values()]
            mean_value = np.nansum(player_values)
            attributes_dict[attribute] = mean_value

        game_dict[team] = attributes_dict
        
    return game_dict

Process

In [ ]:
# Import Data
merged_df = pd.read_csv('../../Data/From_Preparation/Lineup_FifaPlayersAttributes.csv').set_index('id')
# Filter and show
merged_df = merged_df.loc[:, 'home_pl1_lastname':]
print(merged_df.shape)
merged_df.head(2)

(10537, 1122)


,home_pl1_lastname,home_pl1_firstname,home_pl1_ObservationDate,home_pl1_Name,home_pl1_KitNumber,home_pl1_FavoritePosition,home_pl1_Age,home_pl1_Height,home_pl1_Weight,home_pl1_Club,home_pl1_PlayNational,home_pl1_NationalTeam,home_pl1_Value,home_pl1_Wage,home_pl1_BallControl,home_pl1_Marking,home_pl1_Dribbling,home_pl1_SlideTackle,home_pl1_StandTackle,home_pl1_Aggression,home_pl1_Reactions,home_pl1_Att.Position,home_pl1_Interceptions,home_pl1_Vision,home_pl1_Crossing,home_pl1_ShortPass,home_pl1_LongPass,home_pl1_Acceleration,home_pl1_Stamina,home_pl1_Strength,home_pl1_Balance,home_pl1_SprintSpeed,home_pl1_Agility,home_pl1_Jumping,home_pl1_Heading,home_pl1_ShotPower,home_pl1_Finishing,home_pl1_LongShots,home_pl1_Curve,home_pl1_FKAcc.,home_pl1_Penalties,home_pl1_Volleys,home_pl1_GKPositioning,home_pl1_GKDiving,home_pl1_GKHandling,home_pl1_GKKicking,home_pl1_GKReflexes,home_pl1_Overall,home_pl1_Potential,home_pl1_Lastname,home_pl1_Firstname,away_pl1_lastname,away_pl1_firstname,away_pl1_ObservationDate,away_pl1_Name,away_pl1_KitNumber,away_pl1_FavoritePosition,away_pl1_Age,away_pl1_Height,away_pl1_Weight,away_pl1_Club,away_pl1_PlayNational,away_pl1_NationalTeam,away_pl1_Value,away_pl1_Wage,away_pl1_BallControl,away_pl1_Marking,away_pl1_Dribbling,away_pl1_SlideTackle,away_pl1_StandTackle,away_pl1_Aggression,away_pl1_Reactions,away_pl1_Att.Position,away_pl1_Interceptions,away_pl1_Vision,away_pl1_Crossing,away_pl1_ShortPass,away_pl1_LongPass,away_pl1_Acceleration,away_pl1_Stamina,away_pl1_Strength,away_pl1_Balance,away_pl1_SprintSpeed,away_pl1_Agility,away_pl1_Jumping,away_pl1_Heading,away_pl1_ShotPower,away_pl1_Finishing,away_pl1_LongShots,away_pl1_Curve,away_pl1_FKAcc.,away_pl1_Penalties,away_pl1_Volleys,away_pl1_GKPositioning,away_pl1_GKDiving,away_pl1_GKHandling,away_pl1_GKKicking,away_pl1_GKReflexes,away_pl1_Overall,away_pl1_Potential,away_pl1_Lastname,away_pl1_Firstname,home_pl2_lastname,home_pl2_firstname,home_pl2_ObservationDate,home_pl2_Name,home_pl2_KitNumber,home_pl2_FavoritePosition,home_pl2_Age,home_pl2_Height,home_pl2_Weight,home_pl2_Club,home_pl2_PlayNational,home_pl2_NationalTeam,home_pl2_Value,home_pl2_Wage,home_pl2_BallControl,home_pl2_Marking,home_pl2_Dribbling,home_pl2_SlideTackle,home_pl2_StandTackle,home_pl2_Aggression,home_pl2_Reactions,home_pl2_Att.Position,home_pl2_Interceptions,home_pl2_Vision,home_pl2_Crossing,home_pl2_ShortPass,home_pl2_LongPass,home_pl2_Acceleration,home_pl2_Stamina,home_pl2_Strength,home_pl2_Balance,home_pl2_SprintSpeed,home_pl2_Agility,home_pl2_Jumping,home_pl2_Heading,home_pl2_ShotPower,home_pl2_Finishing,home_pl2_LongShots,home_pl2_Curve,home_pl2_FKAcc.,home_pl2_Penalties,home_pl2_Volleys,home_pl2_GKPositioning,home_pl2_GKDiving,home_pl2_GKHandling,home_pl2_GKKicking,home_pl2_GKReflexes,home_pl2_Overall,home_pl2_Potential,home_pl2_Lastname,home_pl2_Firstname,away_pl2_lastname,away_pl2_firstname,away_pl2_ObservationDate,away_pl2_Name,away_pl2_KitNumber,away_pl2_FavoritePosition,away_pl2_Age,away_pl2_Height,away_pl2_Weight,away_pl2_Club,away_pl2_PlayNational,away_pl2_NationalTeam,away_pl2_Value,away_pl2_Wage,away_pl2_BallControl,away_pl2_Marking,away_pl2_Dribbling,away_pl2_SlideTackle,away_pl2_StandTackle,away_pl2_Aggression,away_pl2_Reactions,away_pl2_Att.Position,away_pl2_Interceptions,away_pl2_Vision,away_pl2_Crossing,away_pl2_ShortPass,away_pl2_LongPass,away_pl2_Acceleration,away_pl2_Stamina,away_pl2_Strength,away_pl2_Balance,away_pl2_SprintSpeed,away_pl2_Agility,away_pl2_Jumping,away_pl2_Heading,away_pl2_ShotPower,away_pl2_Finishing,away_pl2_LongShots,away_pl2_Curve,away_pl2_FKAcc.,away_pl2_Penalties,away_pl2_Volleys,away_pl2_GKPositioning,away_pl2_GKDiving,away_pl2_GKHandling,away_pl2_GKKicking,away_pl2_GKReflexes,away_pl2_Overall,away_pl2_Potential,away_pl2_Lastname,away_pl2_Firstname,home_pl3_lastname,home_pl3_firstname,home_pl3_ObservationDate,home_pl3_Name,home_pl3_KitNumber,home_pl3_FavoritePosition,home_pl3_Age,home_pl3_Height,home_pl3_Weight,home_pl3_Club,home_pl3_PlayNation

In [ ]:
# Create the schema for the desired object created in the next steps
grouped_team = merged_df.columns.to_series().groupby(merged_df.columns.str.split('_').str[0]).apply(list)
teams_schema = {}
for i, team_list in enumerate(grouped_team):
    split_strings = [string.split('_') for string in team_list]
    teams_schema[grouped_team.index[i]] = \
        {player: [s[-1] for s in group] for player, group in groupby(sorted(split_strings), key=lambda x: x[1])}

In [ ]:
def create_teams_dict(row):
    
    teams = {}
    # Loop through the teams schema to create the players dictionaries for each team
    for team_pref, players_dict in teams_schema.items():
        players = {}
        # Loop through the players schema to create the attributes dictionaries for each player
        for player_pref, list_attributes in players_dict.items():
            # Use a dictionary comprehension to generate the attributes dictionary for each player
            single_player = {attr: row[team_pref + '_' + player_pref + '_' + attr] for attr in list_attributes}
            players[player_pref] = single_player
        teams[team_pref] = players
        
    return teams

In [ ]:
tqdm.pandas()
teams_dict_list = merged_df.progress_apply(create_teams_dict, axis=1).tolist()
all_games = {index: teams for index, teams in zip(merged_df.index, teams_dict_list)}

  0%|          | 0/10537 [00:00<?, ?it/s]

In [ ]:
all_games_comp = {index: create_comp_game(game) for index, game in all_games.items()}

flattened_dict = {key: flatten(value) for key, value in all_games_comp.items()}
new_df_columns = list(flattened_dict[251728].keys())

final_player_df = pd.DataFrame.from_dict(flattened_dict, orient='index', columns=new_df_columns)
final_player_df.head()

,away_Acceleration,away_Age,away_Aggression,away_Agility,away_Balance,away_Crossing,away_Height,away_Jumping,away_LongPass,away_Overall,away_Potential,away_Reactions,away_ShortPass,away_SprintSpeed,away_Stamina,away_Strength,away_Value,away_Vision,away_Wage,away_Weight,away_Att.Position,away_BallControl,away_Curve,away_Dribbling,away_Finishing,away_Heading,away_LongShots,away_ShotPower,away_Volleys,away_Marking,away_SlideTackle,away_StandTackle,away_Interceptions,away_GKDiving,away_GKHandling,away_GKKicking,away_GKPositioning,away_GKReflexes,away_FKAcc.,away_Penalties,away_PlayNational,home_Acceleration,home_Age,home_Aggression,home_Agility,home_Balance,home_Crossing,home_Height,home_Jumping,home_LongPass,home_Overall,home_Potential,home_Reactions,home_ShortPass,home_SprintSpeed,home_Stamina,home_Strength,home_Value,home_Vision,home_Wage,home_Weight,home_Att.Position,home_BallControl,home_Curve,home_Dribbling,home_Finishing,home_Heading,home_LongShots,home_ShotPower,home_Volleys,home_Marking,home_SlideTackle,home_StandTackle,home_Interceptions,home_GKDiving,home_GKHandling,home_GKKicking,home_GKPositioning,home_GKReflexes,home_FKAcc.,home_Penalties,home_PlayNational
251728,70.2,25.4,69.4,69.6,64.5,63.7,183.4,73.5,65.4,78.4,82.5,74.8,71.9,70.4,71.0,71.3,18113636.4,68.0,107909.1,76.2,73.8,84.6,76.0,82.8,70.4,66.6,77.6,81.0,74.8,71.8,75.4,74.8,70.2,83.0,79.0,79.0,80.0,85.0,76.0,87.0,4.0,59.4,25.7,66.3,61.5,63.6,45.8,180.1,66.2,53.5,67.4,72.5,66.1,60.7,59.6,58.9,64.3,1180000.0,49.9,14590.9,74.8,56.4,68.0,48.4,59.8,57.2,65.2,53.4,61.4,46.0,67.2,71.2,73.0,67.2,71.0,72.0,66.0,72.0,76.0,54.0,59.0,0.0
251738,65.1,26.8,62.5,65.6,62.6,53.9,183.5,73.0,57.2,71.5,74.3,67.7,63.5,68.9,66.8,73.9,2295454.5,55.0,27090.9,77.7,64.2,69.7,56.7,67.5,60.0,69.3,62.0,68.7,57.3,74.2,72.2,74.8,70.2,68.0,63.0,76.0,69.0,75.0,74.0,69.0,1.0,72.5,24.2,62.1,64.9,62.8,52.9,182.9,69.8,58.0,74.2,79.7,69.1,64.3,73.2,67.5,74.5,6556818.2,51.8,38727.3,76.8,64.7,71.2,59.7,70.8,60.2,55.2,61.0,66.2,55.8,78.0,78.5,79.8,80.2,82.0,78.0,79.0,81.0,83.0,74.0,79.0,1.0
252926,69.2,27.8,63.5,66.3,69.5,62.6,178.2,64.3,60.8,74.5,76.7,71.2,69.0,69.6,67.6,66.3,4545454.5,63.8,41181.8,75.0,66.7,76.2,60.0,73.5,65.5,56.2,62.5,70.8,59.2,74.8,72.8,73.8,70.5,78.0,80.0,77.0,78.0,90.0,81.0,75.0,1.0,62.5,26.2,59.5,61.5,60.5,56.5,182.3,64.1,56.5,68.6,72.3,63.4,62.6,62.2,71.9,69.4,1740909.1,52.0,12500.0,78.3,61.0,70.3,60.5,68.0,60.0,60.5,60.2,65.7,57.3,68.5,68.8,69.2,66.2,64.0,71.0,60.0,62.0,65.0,76.0,78.0,1.0
251760,61.0,27.1,63.9,63.2,60.8,53.8,183.1,66.8,57.2,70.9,72.7,67.1,63.4,62.7,70.5,72.4,1956818.2,56.5,22636.4,77.1,69.3,71.0,57.2,68.5,65.8,68.2,62.5,69.0,53.2,73.5,70.5,74.0,73.5,71.0,67.0,56.0,76.0,74.0,64.0,75.0,0.0,67.8,26.8,64.5,68.0,62.4,60.7,181.2,67.1,63.8,72.7,75.6,68.0,68.4,69.3,65.7,70.6,3307000.0,62.4,28500.0,77.6,67.3,73.5,62.0,71.3,70.0,63.0,66.8,73.0,66.2,72.2,76.0,75.2,73.5,80.0,75.0,69.0,76.0,80.0,80.0,87.0,0.0
252536,72.0,25.6,62.8,67.0,65.6,55.2,179.7,67.8,58.9,73.2,75.9,69.6,63.5,70.7,71.1,68.5,3536363.6,58.4,31636.4,75.6,70.7,74.7,59.0,74.0,67.8,61.3,66.0,75.0,66.2,73.0,71.8,75.0,70.0,79.0,73.0,71.0,76.0,82.0,71.0,79.0,0.0,59.1,29.3,63.3,60.8,61.2,52.2,183.5,72.0,57.5,72.1,72.7,69.3,62.4,63.5,65.4,72.3,2009090.9,59.3,24909.1,77.7,70.0,71.2,61.8,69.8,68.6,64.0,67.8,69.6,65.2,73.0,71.8,72.4,69.4,78.0,75.0,76.0,76.0,78.0,73.0,72.0,0.0


In [ ]:
final_player_df.describe()

,away_Acceleration,away_Age,away_Aggression,away_Agility,away_Balance,away_Crossing,away_Height,away_Jumping,away_LongPass,away_Overall,away_Potential,away_Reactions,away_ShortPass,away_SprintSpeed,away_Stamina,away_Strength,away_Value,away_Vision,away_Wage,away_Weight,away_Att.Position,away_BallControl,away_Curve,away_Dribbling,away_Finishing,away_Heading,away_LongShots,away_ShotPower,away_Volleys,away_Marking,away_SlideTackle,away_StandTackle,away_Interceptions,away_GKDiving,away_GKHandling,away_GKKicking,away_GKPositioning,away_GKReflexes,away_FKAcc.,away_Penalties,away_PlayNational,home_Acceleration,home_Age,home_Aggression,home_Agility,home_Balance,home_Crossing,home_Height,home_Jumping,home_LongPass,home_Overall,home_Potential,home_Reactions,home_ShortPass,home_SprintSpeed,home_Stamina,home_Strength,home_Value,home_Vision,home_Wage,home_Weight,home_Att.Position,home_BallControl,home_Curve,home_Dribbling,home_Finishing,home_Heading,home_LongShots,home_ShotPower,home_Volleys,home_Marking,home_SlideTackle,home_StandTackle,home_Interceptions,home_GKDiving,home_GKHandling,home_GKKicking,home_GKPositioning,home_GKReflexes,home_FKAcc.,home_Penalties,home_PlayNational
count,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,1.053700e+04,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.00000,10537.000000,10537.000000,10537.000000,1.053700e+04,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.00000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000
mean,68.046256,26.502116,65.497229,67.303189,65.475354,59.160776,182.661222,69.212746,63.402695,76.426526,79.746285,73.444814,69.339755,68.545288,70.184948,70.541634,1.310971e+07,62.996973,46663.252596,76.815099,71.884426,77.281921,67.245516,75.587103,67.455557,61.874272,69.237639,74.262617,63.596574,74.260511,75.125510,76.596460,74.475211,78.350384,76.000475,72.003701,76.893993,80.065199,77.543418,77.649141,2.557844,68.123280,26.539575,65.448780,67.463747,65.592076,59.270077,182.604261,69.212148,63.450897,76.495188,79.792123,73.519370,69.40707,68.607668,70.151998,70.454797,1.318500e+07,63.153393,46869.541976,76.765977,72.053108,77.382206,67.451495,75.707697,67.682225,61.857123,69.388820,74.359894,63.835864,74.324276,75.185432,76.642431,74.53277,78.419095,76.071273,72.055234,76.974566,80.158489,77.784094,77.864572,2.557939
std,4.006520,1.424881,4.008635,3.916097,3.777812,4.662238,1.893838,3.634794,4.595075,3.766637,3.550819,4.691887,4.133762,4.146409,3.873909,3.188274,1.185541e+07,5.626366,42415.554796,2.298086,5.018482,4.204535,6.394906,4.515853,5.429870,4.946303,5.278641,4.290891,6.186051,4.369938,4.130331,3.878357,4.441230,7.286706,7.659460,8.488272,7.480502,7.556244,6.365649,6.368716,2.366460,3.991231,1.428089,3.988498,3.880806,3.742346,4.608724,1.895094,3.622606,4.548884,3.733829,3.538986,4.642683,4.10636,4.126656,3.868867,3.168537,1.176290e+07,5.571487,42478.036593,2.277787,4.946819,4.169474,6.306831,4.462931,5.348056,4.876436,5.235627,4.238226,6.108033,4.346405,4.117587,3.851543,4.43596,7.199809,7.590153,8.420199,7.370749,7.468850,6.217870,6.314776,2.335533
min,53.700000,21.900000,49.700000,53.500000,52.000000,42.500000,176.000000,54.600000,46.500000,66.600000,69.700000,59.700000,55.300000,54.100000,53.900000

In [ ]:
final_player_df.to_csv('../../Data/Modeling_Before_Preparation/Players_attributes_v01.csv')